In [1]:
#P14 In-Class 15481075 이재근
# 특정한 하나의 예측 방법이 아니라 복수의 예측 모형을 결합하여 더 나은 성능의 예측을 시도해보았다. 모형결합 방법을 통해 단일모형일 떄 보다 성능 분산을 줄일 수 있었던 것 같다.

In [2]:
import findspark

In [3]:
findspark.init('C:\spark-2.4.5-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('treecode').getOrCreate()

In [6]:
data = spark.read.csv('College.csv', inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [8]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [11]:
assembler = VectorAssembler(
    inputCols = ['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol = 'features')

In [12]:
output = assembler.transform(data)

In [13]:
from pyspark.ml.feature import StringIndexer

In [14]:
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex')

In [15]:
output_fixed = indexer.fit(output).transform(output)

In [16]:
final_data = output_fixed.select('features','PrivateIndex')

In [17]:
final_data.show()

+--------------------+------------+
|            features|PrivateIndex|
+--------------------+------------+
|[1660.0,1232.0,72...|         0.0|
|[2186.0,1924.0,51...|         0.0|
|[1428.0,1097.0,33...|         0.0|
|[417.0,349.0,137....|         0.0|
|[193.0,146.0,55.0...|         0.0|
|[587.0,479.0,158....|         0.0|
|[353.0,340.0,103....|         0.0|
|[1899.0,1720.0,48...|         0.0|
|[1038.0,839.0,227...|         0.0|
|[582.0,498.0,172....|         0.0|
|[1732.0,1425.0,47...|         0.0|
|[2652.0,1900.0,48...|         0.0|
|[1179.0,780.0,290...|         0.0|
|[1267.0,1080.0,38...|         0.0|
|[494.0,313.0,157....|         0.0|
|[1420.0,1093.0,22...|         0.0|
|[4302.0,992.0,418...|         0.0|
|[1216.0,908.0,423...|         0.0|
|[1130.0,704.0,322...|         0.0|
|[3540.0,2001.0,10...|         1.0|
+--------------------+------------+
only showing top 20 rows



In [18]:
trainingData, testData = final_data.randomSplit([0.7,0.3])

In [19]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

In [20]:
from pyspark.ml import Pipeline

In [21]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol = 'features')
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol = 'features')
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol = 'features')

In [22]:
dtc_model = dtc.fit(trainingData)
rfc_model = rfc.fit(trainingData)
gbt_model = gbt.fit(trainingData)

In [23]:
dtc_prediction = dtc_model.transform(testData)
rfc_prediction = rfc_model.transform(testData)
gbt_prediction = gbt_model.transform(testData)

In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol='PrivateIndex',predictionCol='prediction',metricName='accuracy')

In [26]:
dtc_acc = acc_evaluator.evaluate(dtc_prediction)
rfc_acc = acc_evaluator.evaluate(rfc_prediction)
gbt_acc = acc_evaluator.evaluate(gbt_prediction)

In [27]:
dtc_acc

0.8968253968253969

In [28]:
rfc_acc

0.9285714285714286

In [29]:
gbt_acc

0.8849206349206349